### Building a Chatbot

We'll go over an example of how to design and implement an LLM-Powered Chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

    * Conversational RAG: Enable a chatbot experience over an external source of data.
    * Agents: Build a chatbot that can take actions.

In [60]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')

In [61]:
from langchain_groq import ChatGroq

model = ChatGroq(model='gemma2-9b-it', groq_api_key=groq_api_key)

In [62]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, My name is Aaron and I'm senior developer in asp.net and learning AI engineer")])

AIMessage(content="Hi Aaron,\n\nThat's great! It's awesome that you're bringing your ASP.NET expertise into the world of AI.  \n\nAs a large language model, I can definitely help you on your journey. What specific areas of AI are you interested in learning about?\n\nFor example, are you looking to:\n\n* **Build AI-powered applications with ASP.NET?**\n* **Understand machine learning concepts and algorithms?**\n* **Explore deep learning and neural networks?**\n* **Get hands-on with AI tools and libraries?**\n\nTell me more about your goals, and I can provide you with relevant resources, explanations, or even code examples to get you started.  \n\nLet's learn together!\n", response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 29, 'total_tokens': 185, 'completion_time': 0.283636364, 'prompt_time': 0.002540723, 'queue_time': 1.1638396869999998, 'total_time': 0.286177087}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', '

In [65]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, My name is Aaron and I'm senior developer in asp.net and learning AI engineer"),
        AIMessage(content="Hi Aaron,\n\nIt's great to meet you!\n\nThat's fantastic that you're transitioning from ASP.NET development into the world of AI engineering. Your background in software development will be incredibly valuable as you learn AI. \n\nWhat aspects of AI are you most interested in exploring? \n\nDo you have any specific projects or goals in mind?\n\nI'm here to help you along your journey.  I can provide information, answer questions, and even help you brainstorm ideas.  Let me know how I can be of assistance!\n"),
        HumanMessage(content="Hey, What's my name, previous work and currently what am doing?")
    ]
)

AIMessage(content="You told me your name is Aaron, that you're a senior developer in ASP.NET, and that you're learning to be an AI engineer.  \n\nIs there anything else you'd like to tell me about your background or current projects? 😊  I'm here to learn more!  \n", response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 169, 'total_tokens': 235, 'completion_time': 0.12, 'prompt_time': 0.010169591, 'queue_time': 0.248288704, 'total_time': 0.130169591}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-19643eb7-4dfc-43cf-bb7d-f8f980cda844-0')

### Message History

We can use a Message Histroy class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [66]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {} # Created Dict

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [67]:
config = {"configurable": {"session_id": "chat1"}}

In [68]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Aaron and I'm senior developer in asp.net and learning AI engineer")],
    config=config
)

In [69]:
response.content

"Hi Aaron,\n\nThat's great! It's awesome that you're making the leap into AI engineering. Your background in ASP.NET development will definitely be an asset as you learn. \n\nWhat specific areas of AI are you most interested in exploring? \n\nI can offer some resources and guidance based on your interests. For example, if you're interested in:\n\n* **Machine Learning:** We could discuss popular frameworks like TensorFlow or PyTorch, and how they can be used to build models for tasks like classification, regression, or natural language processing.\n* **Deep Learning:** I could share information about neural networks, convolutional neural networks (CNNs), and recurrent neural networks (RNNs), which are powerful tools for image recognition, speech processing, and other complex tasks.\n* **Computer Vision:** We could explore techniques for object detection, image segmentation, and image generation.\n* **Natural Language Processing:** I can tell you about techniques for text classification,

In [70]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

AIMessage(content='Your name is Aaron. 😊  You told me at the beginning of our conversation! \n', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 278, 'total_tokens': 299, 'completion_time': 0.038181818, 'prompt_time': 0.015819496, 'queue_time': 0.24792114400000004, 'total_time': 0.054001314}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e5965a12-6e84-4a68-b671-564b1d00708c-0')

In [71]:
# Change the config --> Session id
config1 = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke([HumanMessage(content="What's my name?")], config=config1)

response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know!\n"

### Prompt Templates

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [72]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [73]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Aaron")]})

AIMessage(content="Hi Aaron, it's nice to meet you!\n\nWhat can I do for you today?  I'm happy to answer your questions and help in any way I can. 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 30, 'total_tokens': 73, 'completion_time': 0.078181818, 'prompt_time': 0.002078122, 'queue_time': 0.249723565, 'total_time': 0.08025994}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-dd446b34-9f89-4041-bad1-31dd7269019c-0')

In [75]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [76]:
config = {"configurable": {"session_id": "chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hey my name is Aaron")],
    config=config
)

response.content

"Hello Aaron! It's nice to meet you.  \n\nWhat can I do for you today? 😊 \n"

#### Multiple Input Prompt

In [77]:
# Add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [78]:
response = chain.invoke({"messages": [HumanMessage(content="Hi my name is Aaron")], "language":"tamil"})
response.content

"வணக்கம், ஏரன்! \n\nநான் உதவும் ஒரு உதவிக்குழு. \n\nஎன்னால் முடியும் எனக்கு கேளுங்கள். 😊 \n\n(Hello, Aaron! \n\nI am a helpful assistant.\n\nAsk me anything you want. 😊) \n\n\nLet me know what you'd like to know or do! \n"

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [79]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [80]:
config = {"configurable": {"session_id": "chat4"}}

response = with_message_history.invoke({"messages": [HumanMessage(content="Hi, Im Aaron")], "language":"tamil"}, config=config)
response.content

'வணக்கம் Aaron!  😊\n\nநான் உங்களுக்கு உதவும் ஒரு பயன்பான உதவியாளர். என்னால் உங்கள் கேள்விகளுக்கு சிறந்த முறையில் தமிழில் பதில் அளிக்க முடியும். \n\nஎன்னுடைய உதவியை எப்படி பயன்படுத்தலாம்? 🤔  \n'

## Managing the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

'trim_message' - Helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [82]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

trimmer = trim_messages(
    max_tokens=45,
    token_counter=model,
    strategy="last",
    allow_partial=False,
    start_on="human",
    include_system=True
)

messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="Hi! I'm bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="You're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [83]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | model)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what ice cream do i like?")],
        "language": "English"
    }
)

response.content

"As an AI, I have no memory of past conversations or personal information about you, so I don't know what your favorite ice cream is.\n\nWhat's your favorite flavor?  🍦😊\n"

In [84]:
response = chain.invoke({
    "messages": messages + [HumanMessage(content="What math problem did i ask?")],
    "language": "English"
})

print(response.content)

You asked "What is 2 + 2?".  



Let me know if you have any other questions! 😊



In [85]:
# Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable":{"session_id": "chat6"}}

In [87]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language" : "English"
    },
    config=config
)

response.content

"I don't have access to past conversations, so I don't know your name. Would you like to tell me? 😊  \n\n\n\n\n"